In [93]:
# !pip install pandas
# !pip install sqlalchemy
# !pip install psycopg2-binary
# !pip install mysql-connector-python
# !pip install pyodbc       # DRIVER AZURE PARA MACOS: https://learn.microsoft.com/en-us/sql/connect/odbc/download-odbc-driver-for-sql-server?view=sql-server-ver15#download-for-linux-and-macos

In [259]:
from sqlalchemy import create_engine, inspect, text, Column, Integer, String, TIMESTAMP, ForeignKey, DECIMAL
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
import psycopg2
import pandas as pd
from datetime import datetime as dt
import os
from dotenv import load_dotenv

<br>

# CONEXIÓN CON LA BASE DE DATOS POSTGRES

### Desplegada en Render

In [95]:
load_dotenv()

RENDER_USER = os.getenv('RENDER_USER')
RENDER_PASS = os.getenv('RENDER_PASS')
RENDER_HOST = os.getenv('RENDER_HOST')
RENDER_PORT = os.getenv('RENDER_PORT')

In [96]:
db_url = f'postgresql+psycopg2://{RENDER_USER}:{RENDER_PASS}@{RENDER_HOST}:{RENDER_PORT}/desafio_6wsz'
engine = create_engine(db_url)

### Comprobar que la conexión ha resultado con éxito

In [246]:
inspector = inspect(engine)
table_names = inspector.get_table_names()
table_names

['facilities',
 'locations',
 'organizations',
 'types',
 'users',
 'location_amenities',
 'amenities',
 'events',
 'educational_center',
 'tokens',
 'service_provisions',
 'roles',
 'guests',
 'contracted_services',
 'service_companies',
 'event_users',
 'feedback']

<br>

# POBLAR DB CON DATOS

<br>

### Crear Tablas

In [179]:
# Definir una clase que herede de declarative_base()
Base = declarative_base()

# Definir la tabla utilizando SQLAlchemy
class Facilities(Base):
    __tablename__ = 'facilities'
    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String(255))
    address = Column(String(255))
    createdAt = Column(TIMESTAMP, nullable=False)
    updatedAt = Column(TIMESTAMP, nullable=False)

    # Establecer la relación con la tabla 'Locations'
    location = relationship('locations', back_populates='facility')


class Locations(Base):
    __tablename__ = 'locations'
    id = Column(Integer, primary_key=True, autoincrement=True)
    facility_id = Column(Integer, ForeignKey('facilities.id'), nullable=False)
    meeting_room = Column(String(255))
    capacity = Column(Integer)
    description = Column(String(255))
    createdAt = Column(TIMESTAMP, nullable=False)
    updatedAt = Column(TIMESTAMP, nullable=False)

    # Establecer la relación con la tabla 'Facilities'
    facility = relationship('facilities', back_populates='location')
    loc_amenities = relationship('location_amenities', back_populates='location')
    events = relationship('events', back_populates='location')


class Amenities(Base):
    __tablename__ = 'amenities'
    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String(255))
    createdAt = Column(TIMESTAMP, nullable=False)
    updatedAt = Column(TIMESTAMP, nullable=False)

    loc_amenities = relationship('location_amenities', back_populates='amenity')


class LocationAmenities(Base):
    __tablename__ = 'location_amenities'
    id = Column(Integer, primary_key=True, autoincrement=True)
    location_id = Column(Integer, ForeignKey('locations.id'), nullable=False)
    amenities_id = Column(Integer, ForeignKey('amenities.id'), nullable=False)
    createdAt = Column(TIMESTAMP, nullable=False)
    updatedAt = Column(TIMESTAMP, nullable=False)

    amenity = relationship('amenities', back_populates='loc_amenities')
    location = relationship('locations', back_populates='loc_amenities')


class Events(Base):
    __tablename__ = 'events'
    id = Column(Integer, primary_key=True, autoincrement=True)
    location_id = Column(Integer, ForeignKey('locations.id'), nullable=False)
    dateTime = Column(TIMESTAMP)
    duration_min = Column(Integer)
    type = Column(String(255))
    banner = Column(String(255))
    speaker = Column(String(255))
    description = Column(String(255))
    title = Column(String(255))
    createdAt = Column(TIMESTAMP, nullable=False)
    updatedAt = Column(TIMESTAMP, nullable=False)

    # Establecer la relación con la tabla 'Locations'
    location = relationship('locations', back_populates='event')
    guest = relationship('guests', back_populates='event')
    contracted = relationship('contracted_services', back_populates='event')
    event_user = relationship('event_users', back_populates='events')


class Guests(Base):
    __tablename__ = 'guests'
    id = Column(Integer, primary_key=True, autoincrement=True)
    event_id = Column(Integer, ForeignKey('events.id'), nullable=False)
    name = Column(String(255))
    email = Column(String(255))
    createdAt = Column(TIMESTAMP, nullable=False)
    updatedAt = Column(TIMESTAMP, nullable=False)

    # Establecer la relación con la tabla 'Events'
    event = relationship('events', back_populates='guest')


class ServiceCompanies(Base):
    __tablename__ = 'service_companies'
    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String(255))
    email = Column(String(255))
    tel = Column(Integer)
    speciality = Column(String(255))
    department = Column(String(255))
    createdAt = Column(TIMESTAMP, nullable=False)
    updatedAt = Column(TIMESTAMP, nullable=False)

    contracted = relationship('contracted_services', back_populates='service_company')


class ServiceProvisions(Base):
    __tablename__ = 'service_provisions'
    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String(255))
    description = Column(String(255))
    createdAt = Column(TIMESTAMP, nullable=False)
    updatedAt = Column(TIMESTAMP, nullable=False)

    contracted = relationship('contracted_services', back_populates='service')


class ContractedServices(Base):
    __tablename__ = 'contracted_services'
    id = Column(Integer, primary_key=True, autoincrement=True)
    service_id = Column(Integer, ForeignKey('service_provisions.id'), nullable=False)
    event_id = Column(Integer, ForeignKey('events.id'), nullable=False)
    cost = Column(DECIMAL)
    paid = Column(DECIMAL)
    service_company_id = Column(Integer, ForeignKey('service_companies.id'))
    createdAt = Column(TIMESTAMP, nullable=False)
    updatedAt = Column(TIMESTAMP, nullable=False)

    # Establecer las relaciones con las tablas correspondientes
    service_company = relationship('service_companies', back_populates='contracted')
    service = relationship('service_provisions', back_populates='contracted')
    event = relationship('events', back_populates='contracted')


class EducationalCenter(Base):
    __tablename__ = 'educational_center'
    id = Column(Integer, primary_key=True, autoincrement=True)
    type_id = Column(Integer, ForeignKey('types.id'), nullable=False)
    name = Column(String(255))
    createdAt = Column(TIMESTAMP, nullable=False)
    updatedAt = Column(TIMESTAMP, nullable=False)

    # Establecer la relación con la tabla 'Types'
    typ = relationship('types', back_populates='ed_center')


class Types(Base):
    __tablename__ = 'types'
    id = Column(Integer, primary_key=True, autoincrement=True)
    organization_id = Column(Integer, ForeignKey('organizations.id'), nullable=False)
    name = Column(String(255))
    createdAt = Column(TIMESTAMP, nullable=False)
    updatedAt = Column(TIMESTAMP, nullable=False)

    # Establecer la relación con la tabla 'Organizations'
    ed_center = relationship('educational_center', back_populates='typ')
    organization = relationship('organizations', back_populates='typ')


class Organizations(Base):
    __tablename__ = 'organizations'
    id = Column(Integer, primary_key=True, autoincrement=True)
    type_id = Column(Integer)
    name = Column(String(255))
    web = Column(String(255))
    email = Column(String(255))
    tel = Column(Integer)
    position = Column(String(255))
    sector = Column(String(255))
    createdAt = Column(TIMESTAMP, nullable=False)
    updatedAt = Column(TIMESTAMP, nullable=False)

    typ = relationship('types', back_populates='organization')
    user = relationship('users', back_populates='organization')
    

class Users(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True, autoincrement=True)
    organization_id = Column(Integer, ForeignKey('organizations.id'), nullable=False)
    name = Column(String(255))
    surname = Column(String(255))
    email = Column(String(255))
    password = Column(String(255))
    tel = Column(Integer)
    birthdate = Column(TIMESTAMP)
    avatar = Column(String(255))
    confirmed = Column(String(255))
    file = Column(String(255))
    course = Column(String(255))
    country = Column(String(255))
    interest = Column(String(255))
    gender = Column(String(255))
    catchment = Column(String(255))
    career = Column(String(255))
    responsability = Column(String(255))
    area = Column(String(255))
    createdAt = Column(TIMESTAMP, nullable=False)
    updatedAt = Column(TIMESTAMP, nullable=False)

    # Establecer la relación con la tabla 'Organizations'
    organization = relationship('organizations', back_populates='user')
    token = relationship('tokens', back_populates='user')
    rol = relationship('roles', back_populates='user')
    event_user = relationship('event_users', back_populates='user')


class Tokens(Base):
    __tablename__ = 'tokens'
    id = Column(Integer, primary_key=True, autoincrement=True)
    User_id = Column(Integer, ForeignKey('users.id'), nullable=False)
    token = Column(String(255))
    createdAt = Column(TIMESTAMP, nullable=False)
    updatedAt = Column(TIMESTAMP, nullable=False)

    # Establecer la relación con la tabla 'Users'
    user = relationship('users', back_populates='token')


class Roles(Base):
    __tablename__ = 'roles'
    id = Column(Integer, primary_key=True, autoincrement=True)
    User_id = Column(Integer, ForeignKey('users.id'), nullable=False)
    type = Column(String(255))
    createdAt = Column(TIMESTAMP, nullable=False)
    updatedAt = Column(TIMESTAMP, nullable=False)

    # Establecer la relación con la tabla 'Users'
    user = relationship('users', back_populates='rol')


class EventUsers(Base):
    __tablename__ = 'event_users'
    id = Column(Integer, primary_key=True, autoincrement=True)
    event_id = Column(Integer, ForeignKey('events.id'), nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'), nullable=False)
    arriveTime = Column(TIMESTAMP)
    qrtoken = Column(String(255))
    leaveTime = Column(TIMESTAMP)
    createdAt = Column(TIMESTAMP, nullable=False)
    updatedAt = Column(TIMESTAMP, nullable=False)

    # Establecer las relaciones con las tablas 'Events' y 'Users'
    event = relationship('events', back_populates='event_user')
    user = relationship('users', back_populates='event_user')
    feedb = relationship('feedback', back_populates='event_user')


class Feedbacks(Base):
    __tablename__ = 'feedback'
    id = Column(Integer, primary_key=True, autoincrement=True)
    valoration = Column(Integer)
    eventUser_id = Column(Integer, ForeignKey('event_users.id'), nullable=False)
    comments = Column(String(255))
    createdAt = Column(TIMESTAMP, nullable=False)
    updatedAt = Column(TIMESTAMP, nullable=False)

    # Establecer la relación con la tabla 'EventUsers'
    event_user = relationship('event_users', back_populates='feedb')


# Crear la tabla en la base de datos
Base.metadata.create_all(engine)


/var/folders/r9/xzh1ktf97lnd64tzm93xrk_00000gn/T/ipykernel_32619/2188437066.py:2: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


<br>

### Poblar Facilities

In [197]:
columnas = pd.read_sql_table('facilities', con=engine).columns

facilities_excel = pd.read_csv('../data/excel/Facilities.csv')
facilities_excel['createdAt'] = pd.to_datetime(dt.date(dt.today()))
facilities_excel['updatedAt'] = pd.to_datetime(dt.date(dt.today()))
facilities_excel.columns = columnas
facilities_excel = facilities_excel.drop('id', axis=1)

In [198]:
# facilities_excel.to_sql('facilities', con=engine, if_exists='append', index=False)

3

In [199]:
pd.read_sql_table('facilities', con=engine)

,id,name,address,createdAt,updatedAt
0,1,Edem,"Plaça de L'aigua, s/n, 46024 Valencia",2023-09-27,2023-09-27
1,2,Lanzadera,"Carrer del Moll de la Duana, s/n, 46024 Valencia",2023-09-27,2023-09-27
2,3,Angels,"Sc Puerto, 13, 46024 Valencia",2023-09-27,2023-09-27


<br>

### Poblar Locations

In [200]:
columnas = pd.read_sql_table('locations', con=engine).columns

locations_excel = pd.read_csv('../data/excel/Locations.csv')
locations_excel['createdAt'] = pd.to_datetime(dt.date(dt.today()))
locations_excel['updatedAt'] = pd.to_datetime(dt.date(dt.today()))
locations_excel.columns = columnas
locations_excel = locations_excel.drop('id', axis=1)

In [201]:
# locations_excel.to_sql('locations', con=engine, if_exists='append', index=False)

44

In [202]:
pd.read_sql_table('locations', con=engine)

,id,facility_id,meeting_room,capacity,description,createdAt,updatedAt
0,1,1,P-001,50,"Planta 0, Pasillo 1",2023-09-27,2023-09-27
1,2,1,P-101,50,"Planta 1, Pasillo 1",2023-09-27,2023-09-27
2,3,1,P-201,50,"Planta 2, Pasillo 1",2023-09-27,2023-09-27
3,4,1,A-001,60,"Planta 0, Auditorio 1",2023-09-27,2023-09-27
4,5,1,A-101,60,"Planta 1, Auditorio 1",2023-09-27,2023-09-27
5,6,1,A-102,50,"Planta 1, Auditorio 2",2023-09-27,2023-09-27
6,7,1,A-201,40,"Planta 2, Auditorio 1",2023-09-27,2023-09-27
7,8,1,A-202,40,"Planta 2, Auditorio 2",2023-09-27,2023-09-27
8,9,1,L-101,20,"Planta 1, Aula 1",2023-09-27,2023-09-27
9,10,1,L-102,20,"Planta 1, Aula 2",2023-09-27,2023-09-27


<br>

### Poblar Amenities

In [203]:
columnas = pd.read_sql_table('amenities', con=engine).columns

amenities_excel = pd.read_csv('../data/excel/Amenities.csv')
amenities_excel['createdAt'] = pd.to_datetime(dt.date(dt.today()))
amenities_excel['updatedAt'] = pd.to_datetime(dt.date(dt.today()))
amenities_excel.columns = columnas
amenities_excel = amenities_excel.drop('id', axis=1)

In [204]:
# amenities_excel.to_sql('amenities', con=engine, if_exists='append', index=False)

4

In [205]:
pd.read_sql_table('amenities', con=engine)

,id,name,createdAt,updatedAt
0,1,Butacas,2023-09-27,2023-09-27
1,2,Atril,2023-09-27,2023-09-27
2,3,Pantalla,2023-09-27,2023-09-27
3,4,Proyector,2023-09-27,2023-09-27


<br>

### Poblar Location_amenities

In [206]:
columnas = pd.read_sql_table('location_amenities', con=engine).columns

location_amenities_excel = pd.read_csv('../data/excel/Local_amenities.csv')
location_amenities_excel['createdAt'] = pd.to_datetime(dt.date(dt.today()))
location_amenities_excel['updatedAt'] = pd.to_datetime(dt.date(dt.today()))
location_amenities_excel.columns = columnas
location_amenities_excel = location_amenities_excel.drop('id', axis=1)

In [207]:
# location_amenities_excel.to_sql('location_amenities', con=engine, if_exists='append', index=False)

90

In [208]:
pd.read_sql_table('location_amenities', con=engine)

,id,location_id,amenities_id,createdAt,updatedAt
0,1,1,3,2023-09-27,2023-09-27
1,2,2,3,2023-09-27,2023-09-27
2,3,3,3,2023-09-27,2023-09-27
3,4,4,1,2023-09-27,2023-09-27
4,5,4,2,2023-09-27,2023-09-27
...,...,...,...,...,...
85,86,40,3,2023-09-27,2023-09-27
86,87,41,3,2023-09-27,2023-09-27
87,88,42,3,2023-09-27,2023-09-27
88,89,43,3,2023-09-27,2023-09-27


<br>

### Poblar Tabla de Events

In [209]:
columnas = pd.read_sql_table('events', con=engine).columns
col_pop = columnas.values.tolist().pop(columnas.get_loc('speaker'))
columnas = columnas.drop(col_pop)

events_excel = pd.read_csv('../data/excel/Events.csv')
events_excel['createdAt'] = pd.to_datetime(dt.date(dt.today()))
events_excel['updatedAt'] = pd.to_datetime(dt.date(dt.today()))
events_excel.columns = columnas
events_excel = events_excel.drop('id', axis=1)

In [210]:
# events_excel.to_sql('events', con=engine, if_exists='append', index=False)

6

In [258]:
pd.read_sql_table('events', con=engine)

,id,location_id,dateTime,duration_min,type,banner,speaker,description,title,createdAt,updatedAt
0,1,27,2023-10-23 16:00:00,90,Finanzas,None,None,"Borja Astarloa, director de JP Morgan, habla s...",Planteamientos financieros y económicos,2023-09-27,2023-09-27
1,2,6,2023-11-03 17:30:00,60,Marketing,None,None,Isabel López nos hablará sobre el desarrollo d...,Claves para mejorar la experiencia del cliente...,2023-09-27,2023-09-27
2,3,5,2023-09-29 18:00:00,120,Docencia,None,None,Aprovecha los días que EDEM abre sus puertas p...,Open Day Grado ADE,2023-09-27,2023-09-27
3,4,28,2023-09-15 12:00:00,90,IT,None,None,Sònia Valiente nos dará las claves más importa...,Comunicación en entornos digitales,2023-09-27,2023-09-27
4,5,5,2023-09-01 18:00:00,60,Liderazgo,None,None,Mar Pedroche nos mostrará las aplicaciones y p...,Conocer herramientas que mejoran la productivi...,2023-09-27,2023-09-27
5,6,1,2023-11-20 18:00:00,120,Empleo,None,None,Ven a conocer a una selección de empresas del ...,Foro de Empleo IT,2023-09-27,2023-09-27


<br>

### Poblar Tabla de Guests

In [212]:
columnas = pd.read_sql_table('guests', con=engine).columns

guests_excel = pd.read_csv('../data/excel/Guest.csv')
guests_excel['createdAt'] = pd.to_datetime(dt.date(dt.today()))
guests_excel['updatedAt'] = pd.to_datetime(dt.date(dt.today()))
guests_excel.columns = columnas
guests_excel = guests_excel.drop('id', axis=1)

In [213]:
# guests_excel.to_sql('guests', con=engine, if_exists='append', index=False)

24

In [214]:
pd.read_sql_table('guests', con=engine)

,id,event_id,name,email,createdAt,updatedAt
0,1,1,Antonio García,antoniogarcia@example.com,2023-09-27,2023-09-27
1,2,1,Carmen Martínez,carmenmartinez@example.com,2023-09-27,2023-09-27
2,3,1,Francisco López,franciscolopez@example.com,2023-09-27,2023-09-27
3,4,1,María Pérez,mariaperez@example.com,2023-09-27,2023-09-27
4,5,1,Juan Sánchez,juansanchez@example.com,2023-09-27,2023-09-27
5,6,1,Ana Torres,anatorres@example.com,2023-09-27,2023-09-27
6,7,2,José Ruiz,josruiz@example.com,2023-09-27,2023-09-27
7,8,2,Isabel Gómez,isabelgomez@example.com,2023-09-27,2023-09-27
8,9,2,Manuel Romero,manuelromero@example.com,2023-09-27,2023-09-27
9,10,2,Laura Suárez,laurasuarez@example.com,2023-09-27,2023-09-27


<br>

### Poblar Tabla de Service_companies

In [215]:
columnas = pd.read_sql_table('service_companies', con=engine).columns

service_companies_excel = pd.read_csv('../data/excel/Service_company.csv')
service_companies_excel['createdAt'] = pd.to_datetime(dt.date(dt.today()))
service_companies_excel['updatedAt'] = pd.to_datetime(dt.date(dt.today()))
service_companies_excel.columns = columnas
service_companies_excel = service_companies_excel.drop('id', axis=1)

In [216]:
# service_companies_excel.to_sql('service_companies', con=engine, if_exists='append', index=False)

9

In [217]:
pd.read_sql_table('service_companies', con=engine)

,id,name,email,tel,speciality,department,createdAt,updatedAt
0,1,Colevisa,colevisa@colevisa.com,963169106,catering,Ventas,2023-09-27,2023-09-27
1,2,Salsia,info@salsia.es,960089454,catering,Ventas,2023-09-27,2023-09-27
2,3,JMC Fotografos,info@jmc.es,963363914,fotografia,Ventas,2023-09-27,2023-09-27
3,4,Estudio fotografico Lorsil,fotolorsil@gmail.com,963345422,fotografia,Ventas,2023-09-27,2023-09-27
4,5,MA's Events,masevents@gmail.com,662332287,montadores,Ventas,2023-09-27,2023-09-27
5,6,EV-Events,evevents@gmail.com,661894202,montadores,Ventas,2023-09-27,2023-09-27
6,7,Unanime Creativos,creativos@unanime.com,963899098,diseño,Ventas,2023-09-27,2023-09-27
7,8,Agencia Marketing Digital,markdig@agenciamk.com,606817508,diseño,Ventas,2023-09-27,2023-09-27
8,9,Edem - Logística,infoedem@edem.es,963531065,educación,Logística,2023-09-27,2023-09-27


<br>

### Poblar Tabla de Service_Provisions

In [218]:
columnas = pd.read_sql_table('service_provisions', con=engine).columns

service_Provisions_excel = pd.read_csv('../data/excel/Service_Provision.csv')
service_Provisions_excel['createdAt'] = pd.to_datetime(dt.date(dt.today()))
service_Provisions_excel['updatedAt'] = pd.to_datetime(dt.date(dt.today()))
service_Provisions_excel.columns = columnas
service_Provisions_excel = service_Provisions_excel.drop('id', axis=1)

In [219]:
# service_Provisions_excel.to_sql('service_provisions', con=engine, if_exists='append', index=False)

5

In [220]:
pd.read_sql_table('service_provisions', con=engine)

,id,name,description,createdAt,updatedAt
0,1,Servicio de cátering,"Servicio de café, refrescos y bocadillos",2023-09-27,2023-09-27
1,2,Cobertura Logística,Servicio de recepción y apoyo a incidencias IT,2023-09-27,2023-09-27
2,3,Material gráfico,"Elaboración de Flyer y cartel 0,90 x 1,80 cm",2023-09-27,2023-09-27
3,4,Monaje de stands,"Servicio, montaje, desmontaje y retirada de st...",2023-09-27,2023-09-27
4,5,Fotografía,Reportaje fotográfico de evento de duración má...,2023-09-27,2023-09-27


<br>

### Poblar Tabla de Contracted_services

In [221]:
columnas = pd.read_sql_table('contracted_services', con=engine).columns

contracted_services_excel = pd.read_csv('../data/excel/Contracted_Service.csv')
contracted_services_excel['Cost'] = contracted_services_excel['Cost'].str.replace(',', '.').astype(float)
contracted_services_excel['Paid'] = contracted_services_excel['Paid'].str.replace(',', '.').astype(float)
pop_col = contracted_services_excel.pop('ServiceCompany_id')
contracted_services_excel['ServiceCompany_id'] = pop_col
contracted_services_excel['createdAt'] = pd.to_datetime(dt.date(dt.today()))
contracted_services_excel['updatedAt'] = pd.to_datetime(dt.date(dt.today()))
contracted_services_excel.columns = columnas
contracted_services_excel = contracted_services_excel.drop('id', axis=1)

In [224]:
# contracted_services_excel.to_sql('contracted_services', con=engine, if_exists='append', index=False)

20

In [225]:
pd.read_sql_table('contracted_services', con=engine)

,id,service_id,event_id,cost,paid,service_company_id,createdAt,updatedAt
0,1,1,1,200.0,NaN,1,2023-09-27,2023-09-27
1,2,2,1,NaN,NaN,9,2023-09-27,2023-09-27
2,3,3,1,350.0,NaN,7,2023-09-27,2023-09-27
3,4,5,1,300.0,NaN,3,2023-09-27,2023-09-27
4,5,1,2,200.0,NaN,1,2023-09-27,2023-09-27
5,6,2,2,NaN,NaN,9,2023-09-27,2023-09-27
6,7,3,2,350.0,NaN,7,2023-09-27,2023-09-27
7,8,1,3,200.0,NaN,1,2023-09-27,2023-09-27
8,9,2,3,NaN,NaN,9,2023-09-27,2023-09-27
9,10,3,3,350.0,NaN,7,2023-09-27,2023-09-27


<br>

### Poblar Tabla de Organizations

In [226]:
columnas = pd.read_sql_table('organizations', con=engine).columns

organizations_excel = pd.read_csv('../data/excel/Organization.csv', index_col=0)
organizations_excel['createdAt'] = pd.to_datetime(dt.date(dt.today()))
organizations_excel['updatedAt'] = pd.to_datetime(dt.date(dt.today()))
organizations_excel.columns = columnas
organizations_excel = organizations_excel.drop('id', axis=1)

In [227]:
# organizations_excel.to_sql('organizations', con=engine, if_exists='append', index=False)

198

In [228]:
pd.read_sql_table('organizations', con=engine)

,id,type_id,name,web,email,tel,position,sector,createdAt,updatedAt
0,1,44,Lanzadera,https://lanzadera.es,info@lanzadera.es,963568585,General,Emprendimiento,2023-09-27,2023-09-27
1,2,190,Edem,https://edem.eu,info@edem.es,963531065,General,Educación,2023-09-27,2023-09-27
2,3,57,Angels Capital,https://www.angelscapital.es,info@angelscapital.es,963568585,General,Inversión,2023-09-27,2023-09-27
3,4,88,AAA,https://www.triplea.edem.es,triplea@edem.es,963531065,General,organización sin ánimo de lucro,2023-09-27,2023-09-27
4,5,170,ACS Actividades de Construccion y Servicios SA,https://www.grupoacs.com,acs@empresa.com,913439200,General,Industriales,2023-09-27,2023-09-27
...,...,...,...,...,...,...,...,...,...,...
193,194,117,Colegio San Antonio,https://colegiosanantonio.com,info@colegiosanantonio.com,964886746,General,Educación,2023-09-27,2023-09-27
194,195,82,Colegio Nuestra Señora de los Desamparados,https://colegionuestrasenoradelosdesamparados.com,info@colegionuestrasenoradelosdesamparados.com,964091976,General,Educación,2023-09-27,2023-09-27
195,196,41,Universidad Rey Juan Carlos,https://universidadreyjuancarlos.com,info@universidadreyjuancarlos.com,963954427,General,Educación,2023-09-27,2023-09-27
196,197,58,Universidad Autónoma de Madrid,https://universidadautonomademadrid.com,info@universidadautonomademadrid.com,964208403,General,Educación,2023-09-27,2023-09-27


<br>

### Poblar Tabla de Types

In [229]:
columnas = pd.read_sql_table('types', con=engine).columns

types_excel = pd.read_csv('../data/excel/Type.csv')
types_excel['createdAt'] = pd.to_datetime(dt.date(dt.today()))
types_excel['updatedAt'] = pd.to_datetime(dt.date(dt.today()))
types_excel.columns = columnas
types_excel = types_excel.drop('id', axis=1)

In [230]:
# types_excel.to_sql('types', con=engine, if_exists='append', index=False)

198

In [231]:
pd.read_sql_table('types', con=engine)

,id,organization_id,name,createdAt,updatedAt
0,1,1,Marina de Empresas,2023-09-27,2023-09-27
1,2,2,Marina de Empresas,2023-09-27,2023-09-27
2,3,3,Marina de Empresas,2023-09-27,2023-09-27
3,4,4,Marina de Empresas,2023-09-27,2023-09-27
4,5,5,Otro,2023-09-27,2023-09-27
...,...,...,...,...,...
193,194,194,Centro Educativo,2023-09-27,2023-09-27
194,195,195,Centro Educativo,2023-09-27,2023-09-27
195,196,196,Centro Educativo,2023-09-27,2023-09-27
196,197,197,Centro Educativo,2023-09-27,2023-09-27


<br>

### Poblar Tabla de EducationalCenter

In [232]:
columnas = pd.read_sql_table('educational_center', con=engine).columns

educational_center_excel = pd.read_csv('../data/excel/EducationalCenter.csv')
educational_center_excel['createdAt'] = pd.to_datetime(dt.date(dt.today()))
educational_center_excel['updatedAt'] = pd.to_datetime(dt.date(dt.today()))
educational_center_excel.columns = columnas
educational_center_excel = educational_center_excel.drop('id', axis=1)

In [233]:
# educational_center_excel.to_sql('educational_center', con=engine, if_exists='append', index=False)

26

In [234]:
pd.read_sql_table('educational_center', con=engine)

,id,type_id,name,createdAt,updatedAt
0,1,175,Colegio,2023-09-27,2023-09-27
1,2,177,Colegio,2023-09-27,2023-09-27
2,3,178,Colegio,2023-09-27,2023-09-27
3,4,180,Colegio,2023-09-27,2023-09-27
4,5,181,Colegio,2023-09-27,2023-09-27
5,6,182,Colegio,2023-09-27,2023-09-27
6,7,184,Colegio,2023-09-27,2023-09-27
7,8,188,Colegio,2023-09-27,2023-09-27
8,9,189,Colegio,2023-09-27,2023-09-27
9,10,190,Colegio,2023-09-27,2023-09-27


<br>

### Poblar Tabla de Users

In [235]:
columnas = pd.read_sql_table('users', con=engine).columns
col_pop_1 = columnas.values.tolist().pop(columnas.get_loc('confirmed'))
col_pop_2 = columnas.values.tolist().pop(columnas.get_loc('area'))
columnas = columnas.drop([col_pop_1, col_pop_2])

users_excel = pd.read_csv('../data/excel/User.csv')
users_excel['createdAt'] = pd.to_datetime(dt.date(dt.today()))
users_excel['updatedAt'] = pd.to_datetime(dt.date(dt.today()))
users_excel = users_excel.rename({
    'Id': 'id', 
    'Organization_id': 'organization_id', 
    'Name': 'name', 
    'Surname': 'surname', 
    'Email': 'email', 
    'Password': 'password', 
    'Tel': 'tel', 
    'Birthdate': 'birthdate', 
    'Avatar': 'avatar', 
    'Country': 'country', 
    'Gender': 'gender', 
    'Responsability': 'responsability', 
    'Career': 'career', 'Course': 'course', 
    'Catchment': 'catchment', 
    'File': 'file', 
    'Interests': 'interest', 
    'createdAt': 'createdAt', 
    'updatedAt': 'updatedAt'}, axis=1)
users_excel = users_excel.drop('id', axis=1)

In [236]:
# users_excel.to_sql('users', con=engine, if_exists='append', index=False)

60

In [237]:
pd.read_sql_table('users', con=engine).iloc[:, :-1]

,id,organization_id,name,surname,email,password,tel,birthdate,avatar,confirmed,file,course,country,interest,gender,catchment,career,responsability,area,createdAt
0,1,2,Raquel,Cabrera,raquel.cabrera@gmail.com,None,657042370,1980-10-11,None,None,None,None,Croatia,Emprendimiento,Female,Contratado,Grado,Campus manager,None,2023-09-27
1,2,2,Manuel,Torres,manuel.torres@gmail.com,None,628069713,1981-03-05,None,None,None,None,Spain,Inversión,Male,Contratado,Máster,Gerente de Logística,None,2023-09-27
2,3,2,Manuel,Johnson,manuel.johnson@gmail.com,None,612472809,1982-11-12,None,None,None,None,Spain,Docencia,Male,Contratado,Máster,Gerente de Logística,None,2023-09-27
3,4,2,Diego,Hernández,diego.hernández@gmail.com,None,661070823,1979-08-28,None,None,None,None,Spain,Nuevas tecnologías,Male,Contratado,Grado,Gerente de Marketing,None,2023-09-27
4,5,2,Mario,Molina,mario.molina@gmail.com,None,660731374,1985-09-15,None,None,None,None,Norway,Emprendimiento,Male,Contratado,Máster,Gerente de Marketing,None,2023-09-27
5,6,2,Javier,Thomas,javier.thomas@gmail.com,None,629968039,1978-11-30,None,None,None,None,Spain,Emprendimiento,Male,Contratado,Máster,Ponente,None,2023-09-27
6,7,2,Fernando,Herrero,fernando.herrero@gmail.com,None,629415962,1984-06-22,None,None,None,None,Spain,Docencia,Male,Contratado,Grado,profesores,None,2023-09-27
7,8,2,Gabriel,Suárez,gabriel.suárez@gmail.com,None,614253808,1985-03-05,None,None,None,None,Spain,Finanzas,Male,Contratado,Grado,profesores,None,2023-09-27
8,9,2,Ana,Vega,ana.vega@gmail.com,None,688536659,1986-11-12,None,None,None,None,Ireland,Derecho,Female,Contratado,Grado,Responsable de Área Bootcamps,None,2023-09-27
9,10,2,Mario,Vicente,mario.vicente@gmail.com,None,616824481,1983-08-28,None,None,None,None,Spain,Recursos humanos,Male,Contratado,Grado,Responsable de Área grado ADE,None,2023-09-27


<br>

### Poblar Tabla de Tokens 

In [238]:
columnas = pd.read_sql_table('tokens', con=engine).columns

tokens_excel = pd.read_csv('../data/excel/Tokens.csv', index_col=0)
tokens_excel['createdAt'] = pd.to_datetime(dt.date(dt.today()))
tokens_excel['updatedAt'] = pd.to_datetime(dt.date(dt.today()))
tokens_excel = tokens_excel.drop('id', axis=1)

In [239]:
# tokens_excel.to_sql('tokens', con=engine, if_exists='append', index=False)

15

In [240]:
pd.read_sql_table('tokens', con=engine)

,id,User_id,token,createdAt,updatedAt
0,1,7,eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpZCI6N...,2023-09-27,2023-09-27
1,2,7,eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpZCI6N...,2023-09-27,2023-09-27
2,3,7,eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpZCI6N...,2023-09-27,2023-09-27
3,4,7,eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpZCI6N...,2023-09-27,2023-09-27
4,5,7,eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpZCI6N...,2023-09-27,2023-09-27
5,6,7,eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpZCI6N...,2023-09-27,2023-09-27
6,7,7,eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpZCI6N...,2023-09-27,2023-09-27
7,8,7,eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpZCI6N...,2023-09-27,2023-09-27
8,9,7,eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpZCI6N...,2023-09-27,2023-09-27
9,10,7,eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpZCI6N...,2023-09-27,2023-09-27


<br>

### Poblar Tabla de Roles

In [241]:
columnas = pd.read_sql_table('roles', con=engine).columns

roles_excel = pd.read_csv('../data/excel/Role.csv')
roles_excel['createdAt'] = pd.to_datetime(dt.date(dt.today()))
roles_excel['updatedAt'] = pd.to_datetime(dt.date(dt.today()))
roles_excel.columns = columnas
roles_excel = roles_excel.drop('id', axis=1)

In [242]:
# roles_excel.to_sql('roles', con=engine, if_exists='append', index=False)

60

In [243]:
pd.read_sql_table('roles', con=engine)

,id,User_id,type,createdAt,updatedAt
0,1,1,User,2023-09-27,2023-09-27
1,2,2,User,2023-09-27,2023-09-27
2,3,3,User,2023-09-27,2023-09-27
3,4,4,Admin,2023-09-27,2023-09-27
4,5,5,Admin,2023-09-27,2023-09-27
5,6,6,User,2023-09-27,2023-09-27
6,7,7,User,2023-09-27,2023-09-27
7,8,8,User,2023-09-27,2023-09-27
8,9,9,Admin,2023-09-27,2023-09-27
9,10,10,Admin,2023-09-27,2023-09-27


<br>

### Poblar Tabla de EventUsers

In [257]:
columnas = pd.read_sql_table('event_users', con=engine).columns

eventUsers_excel = pd.read_csv('../data/excel/Event_User.csv')
eventUsers_excel['createdAt'] = pd.to_datetime(dt.date(dt.today()))
eventUsers_excel['updatedAt'] = pd.to_datetime(dt.date(dt.today()))
eventUsers_excel.columns = columnas
eventUsers_excel = eventUsers_excel.drop('id', axis=1)

In [255]:
eventUsers_excel.to_sql('event_users', con=engine, if_exists='append', index=False)

248

In [256]:
pd.read_sql_table('event_users', con=engine)

,id,event_id,user_id,arriveTime,qrtoken,leaveTime,createdAt,updatedAt
0,1,1,2,NaT,None,NaT,2023-09-27,2023-09-27
1,2,1,4,NaT,None,NaT,2023-09-27,2023-09-27
2,3,1,7,NaT,None,NaT,2023-09-27,2023-09-27
3,4,1,8,NaT,None,NaT,2023-09-27,2023-09-27
4,5,1,9,NaT,None,NaT,2023-09-27,2023-09-27
...,...,...,...,...,...,...,...,...
243,244,6,53,NaT,None,NaT,2023-09-27,2023-09-27
244,245,6,54,NaT,None,NaT,2023-09-27,2023-09-27
245,246,6,58,NaT,None,NaT,2023-09-27,2023-09-27
246,247,6,59,NaT,None,NaT,2023-09-27,2023-09-27


<br>

### Poblar Tabla de Feedback

In [ ]:
pd.read_sql_table('feedback', con=engine)

,id,valoration,eventUser_id,comments,createdAt,updatedAt


<br>

# Observar todas las tablas

In [ ]:
tables = ['events', 'locations', 'facilities', 'location_amenities', 'amenities', 
          'contracted_services', 'service_provisions', 'service_companies', 
          'event_users', 'guests', 'feedback', 
          'users', 'tokens', 'roles', 
          'organizations', 'types', 'educational_center']
dict_tables = {}

for table in tables:
    dict_tables[table] = pd.read_sql_table(table, con=engine)

for table in dict_tables.keys():
    display(dict_tables[table][:5])

print(len(tables))

,id,location_id,dateTime,duration_min,type,banner,speaker,description,title,createdAt,updatedAt
0,1,27,2023-10-23 16:00:00,90,Finanzas,None,None,"Borja Astarloa, director de JP Morgan, habla s...",Planteamientos financieros y económicos,2023-09-27,2023-09-27
1,2,6,2023-11-03 17:30:00,60,Marketing,None,None,Isabel López nos hablará sobre el desarrollo d...,Claves para mejorar la experiencia del cliente...,2023-09-27,2023-09-27
2,3,5,2023-09-29 18:00:00,120,Docencia,None,None,Aprovecha los días que EDEM abre sus puertas p...,Open Day Grado ADE,2023-09-27,2023-09-27
3,4,28,2023-09-15 12:00:00,90,IT,None,None,Sònia Valiente nos dará las claves más importa...,Comunicación en entornos digitales,2023-09-27,2023-09-27
4,5,5,2023-09-01 18:00:00,60,Liderazgo,None,None,Mar Pedroche nos mostrará las aplicaciones y p...,Conocer herramientas que mejoran la productivi...,2023-09-27,2023-09-27


,id,facility_id,meeting_room,capacity,description,createdAt,updatedAt
0,1,1,P-001,50,"Planta 0, Pasillo 1",2023-09-27,2023-09-27
1,2,1,P-101,50,"Planta 1, Pasillo 1",2023-09-27,2023-09-27
2,3,1,P-201,50,"Planta 2, Pasillo 1",2023-09-27,2023-09-27
3,4,1,A-001,60,"Planta 0, Auditorio 1",2023-09-27,2023-09-27
4,5,1,A-101,60,"Planta 1, Auditorio 1",2023-09-27,2023-09-27


,id,name,address,createdAt,updatedAt
0,1,Edem,"Plaça de L'aigua, s/n, 46024 Valencia",2023-09-27,2023-09-27
1,2,Lanzadera,"Carrer del Moll de la Duana, s/n, 46024 Valencia",2023-09-27,2023-09-27
2,3,Angels,"Sc Puerto, 13, 46024 Valencia",2023-09-27,2023-09-27


,id,location_id,amenities_id,createdAt,updatedAt
0,1,1,3,2023-09-27,2023-09-27
1,2,2,3,2023-09-27,2023-09-27
2,3,3,3,2023-09-27,2023-09-27
3,4,4,1,2023-09-27,2023-09-27
4,5,4,2,2023-09-27,2023-09-27


,id,name,createdAt,updatedAt
0,1,Butacas,2023-09-27,2023-09-27
1,2,Atril,2023-09-27,2023-09-27
2,3,Pantalla,2023-09-27,2023-09-27
3,4,Proyector,2023-09-27,2023-09-27


...

,id,User_id,type,createdAt,updatedAt
0,61,1,User,2023-09-27,2023-09-27
1,62,2,User,2023-09-27,2023-09-27
2,63,3,User,2023-09-27,2023-09-27
3,64,4,Admin,2023-09-27,2023-09-27
4,65,5,Admin,2023-09-27,2023-09-27


,id,name,web,email,tel,position,sector,createdAt,updatedAt
0,1,Lanzadera,https://lanzadera.es,info@lanzadera.es,963568585,General,Emprendimiento,2023-09-27,2023-09-27
1,2,Edem,https://edem.eu,info@edem.es,963531065,General,Educación,2023-09-27,2023-09-27
2,3,Angels Capital,https://www.angelscapital.es,info@angelscapital.es,963568585,General,Inversión,2023-09-27,2023-09-27
3,4,AAA,https://www.triplea.edem.es,triplea@edem.es,963531065,General,organización sin ánimo de lucro,2023-09-27,2023-09-27
4,5,ACS Actividades de Construccion y Servicios SA,https://www.grupoacs.com,acs@empresa.com,913439200,General,Industriales,2023-09-27,2023-09-27


,id,organization_id,name,createdAt,updatedAt
0,1,1,Marina de Empresas,2023-09-27,2023-09-27
1,2,2,Marina de Empresas,2023-09-27,2023-09-27
2,3,3,Marina de Empresas,2023-09-27,2023-09-27
3,4,4,Marina de Empresas,2023-09-27,2023-09-27
4,5,5,Otro,2023-09-27,2023-09-27


,id,type_id,name,createdAt,updatedAt
0,1,175,Colegio,2023-09-27,2023-09-27
1,2,177,Colegio,2023-09-27,2023-09-27
2,3,178,Colegio,2023-09-27,2023-09-27
3,4,180,Colegio,2023-09-27,2023-09-27
4,5,181,Colegio,2023-09-27,2023-09-27


17
